L’application utilisée par le banquier nous envoie les informations suivantes:

{
    "RevolvingUtilizationOfUnsecuredLines": 0.01703559,
    "NumberOfDependents": 1,
    "DebtRatio": 0,
    "age": 42,
    "NumberOfOpenCreditLinesAndLoans": 6,
    "NumberRealEstateLoansOrLines": 1,
    "NumberOfTime30-59DaysPastDueNotWorse": 1,
    "NumberOfTime60-89DaysPastDueNotWorse": 0,
    "NumberOfTimes90DaysLate": 0
}

* Créez un nouveau notebook, dans lequel vous calculerez une probabilité de défaut de remboursement.
* Committez, et envoyez moi le lien pour que je retrouve facilement

- Create a new test set with new data from one input
- Do all modifications (same as the train_set)
- BigMl:
  - create sources
  - create datasets
  - train a model (ensemble) on the train_set
  - prediction on the new test_set
  - show the prediction

In [109]:
from pandas import read_csv
import pandas as pd



In [81]:
# Introduce data from 1 input
data = [{'Id': 1,
         'RevolvingUtilizationOfUnsecuredLines': 0.01703559, 
         'age': 42, 
         'NumberOfTime30-59DaysPastDueNotWorse': 1, 
         'DebtRatio': 0, 
         'MonthlyIncome': 0, 
         'NumberOfOpenCreditLinesAndLoans': 6, 
         'NumberOfTimes90DaysLate': 0, 
         'NumberRealEstateLoansOrLines': 1, 
         'NumberOfTime60-89DaysPastDueNotWorse': 0, 
         'NumberOfDependents': 1,
         'SeriousDlqin2yrs': "",
         'NumberCreditLines': 5}]
data

[{'Id': 1,
  'RevolvingUtilizationOfUnsecuredLines': 0.01703559,
  'age': 42,
  'NumberOfTime30-59DaysPastDueNotWorse': 1,
  'DebtRatio': 0,
  'MonthlyIncome': 0,
  'NumberOfOpenCreditLinesAndLoans': 6,
  'NumberOfTimes90DaysLate': 0,
  'NumberRealEstateLoansOrLines': 1,
  'NumberOfTime60-89DaysPastDueNotWorse': 0,
  'NumberOfDependents': 1,
  'SeriousDlqin2yrs': '',
  'NumberCreditLines': 5}]

In [82]:
# Create the table test_set
df_exo = pd.DataFrame(data, columns = ['Id',
 'RevolvingUtilizationOfUnsecuredLines',
 'age',
 'NumberOfTime30-59DaysPastDueNotWorse',
 'DebtRatio',
 'MonthlyIncome',
 'NumberOfOpenCreditLinesAndLoans',
 'NumberOfTimes90DaysLate',
 'NumberRealEstateLoansOrLines',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'NumberOfDependents',
 'SeriousDlqin2yrs',
 'NumberCreditLines'])
df_exo

,Id,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs,NumberCreditLines
0,1,0.017036,42,1,0,0,6,0,1,0,1,,5


In [83]:
# Modifications on the test_set
for row in [df_exo] :
    row['MonthlyIncome'].fillna(0,inplace=True)
    row['NumberOfDependents'].fillna(0,inplace=True)
    row['NumberOfTotalTime'] = row['NumberOfTime30-59DaysPastDueNotWorse'] + row ['NumberOfTimes90DaysLate'] + row['NumberOfTime60-89DaysPastDueNotWorse']
    row['MonthlyDebt'] = row ['DebtRatio'] * row ['MonthlyIncome']
    row['MonthlyBalance'] = row ['MonthlyIncome'] - row ['MonthlyDebt']
    row['RatioIncomeDependents'] = row['MonthlyIncome'] / row ['NumberOfDependents']
df_exo

,Id,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs,NumberCreditLines,NumberOfTotalTime,MonthlyDebt,MonthlyBalance,RatioIncomeDependents
0,1,0.017036,42,1,0,0,6,0,1,0,1,,5,1,0,0,0.0


In [84]:
export_csv = df_exo.to_csv (r'exo_testset.csv', header=True)

In [85]:
import bigml.api
import bigml.api
from bigml.api import BigML
api= BigML (project='project/5d94a52e5a213962e2000331')

In [106]:
# BigML prediction
source = api.create_source('fulltrain_dataframe.csv')
api.ok(source)
dataset_train = api.create_dataset(source)
api.ok(dataset_train)
test_source = api.create_source('exo_testset.csv')
api.ok(test_source)
dataset_test = api.create_dataset(test_source)
api.ok(dataset_test)
model = api.create_ensemble(dataset_train, {"objective_field":"SeriousDlqin2yrs"})
api.ok(model)
batch_prediction = api.create_batch_prediction(model, dataset_test, {"all_fields": True, "confidence": True, "probabilities": True})
api.ok(batch_prediction)
api.download_batch_prediction(batch_prediction, filename='exo_prediction.csv')
df_exo_final = pd.read_csv('exo_prediction.csv', index_col = 0)
df_exo_final

,Id,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,...,SeriousDlqin2yrs,NumberCreditLines,NumberOfTotalTime,MonthlyDebt,MonthlyBalance,RatioIncomeDependents,SeriousDlqin2yrs.1,confidence,0 probability,1 probability
field1,,,,,,,,,,,,,,,,,,,,,
0,1,0.017036,42,1,0,0,6,0,1,0,...,NaN,5,1,0,0,0,0,0.96608,0.9746,0.0254
